In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import numpy as np
import nltk
import plac
import random
from pathlib import Path
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [2]:
def lemmatise_text(text):
    tweet_nlp = nlp(text)
    l = [token.lemma_ for token in tweet_nlp ]
    return (" ".join(l)).replace(" \n","")

def stem_text(text):
    words = word_tokenize(text)
    l = [stemmer.stem(token) for token in words]
    return " ".join(l)


In [3]:
nlp = spacy.load("C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5")

In [4]:
stemmer = SnowballStemmer('french')

In [5]:
FAQ_disney = pd.DataFrame(pd.read_pickle("C:/Users/Benco/Documents/chatBot_disney/faq.pkl"))
FAQ_disney["target"] = 1
FAQ_disney["question"] = FAQ_disney["question"].apply(lambda text : text.replace("\xa0" , ""))
FAQ_disney["reponse"] = FAQ_disney["reponse"].apply(lambda text : text.replace("\xa0" , ""))

In [6]:
html = requests.get("https://www.nike.com/fr/fr_fr/c/kids/how-to-buy-kids-shoes-faq").text
soup = BeautifulSoup(html, 'lxml')
questions = soup.findAll("span",
                      {"class": "nike-cq-subtitle-line-1 nike-cq-title-line nike-cq-line1 nsg-text--dark-grey nike-cq-font22px nike-cq-spacing08px nsg-font-family--platform"}
                        )
questions = [quest.text.replace("\xa0" , "") for quest in questions]
responses = [div.find("p").text for div in soup.findAll("div", { "data-component-container" : "true" , "class"  : "nike-cq-text"})]
responses = list(map(lambda resp : resp.replace("\n" , "").strip().replace("  ","").replace("\xa0","") , responses))
FAQ_nike = pd.DataFrame([{"question" : quest , "reponse" : rep} for quest , rep in zip(questions , responses)])
FAQ_nike["target"] = 0

In [7]:
html_free = requests.get("https://mobile.free.fr/assistance/163.html").text
soup_free = BeautifulSoup(html_free, 'lxml')
question_free = [q.text.replace("\xa0" , "") for q in soup_free.select("li.clicMe > strong")]
rep_free = [r.text.strip().replace("\xa0","").replace("\n","").replace("\t","") for r in soup_free.select("li.hideMe")]
FAQ_free = pd.DataFrame([{"question" : quest , "reponse" : rep} for quest , rep in zip(question_free , rep_free)])
FAQ_free["target"] = 0

In [8]:
train = (FAQ_disney.append(FAQ_nike)).append(FAQ_free)

In [9]:
train["lemmas_rep"] = train["reponse"].apply(lemmatise_text)
train["lemmas_quest"] = train["question"].apply(lemmatise_text)


In [10]:
train["stem_rep"] = train["reponse"].apply(stem_text)
train["stem_quest"] = train["question"].apply(stem_text)

In [ ]:
train.to_csv("train.csv" , index = False)

In [5]:
train = pd.read_csv("train.csv")

In [17]:
for tr in train["stem_quest"]:
    print(tr)

comment postul pour un travail à disneyland paris ?
je possed actuel un passeport annuel classic , fantasy ou dream et je souhait l ’ échang contr un nouveau pass annuel , existe-t-il un prix au prorat de jour rest ?
avec le billet privileg , l ’ offre billet amis est-el amen à disparaîtr ?
avec un pass annuel infinity , puis-j fair bénéfici de l'espac privilégi pour la parad et le show disney illumin à me invit billet privileg , billet amis , autr détenteur de pass annuel ou autr ticket ?
combien de mois sont offert pour le parrainag de 3 filleul effectu et valid avant le 28 mar 2017 et sur quel pass annuel ?
comment déclar un chang d'adress ou autr situat ?
comment fêt un anniversair à disneyland paris ?
comment le fastpass disneyland paris peut-il m'évit de fair la queu ?
comment me rendr aux parc disney depuis mon hôtel ?
comment me rendr aux parc disney depuis mon hôtel ?
comment puis-j me procur de ticket fastpassl jour de ma visit aux parc et diminu le temp d ’ attent dan certai

In [6]:
def localisation(texte , words , label):
    pos = texte.index(words)
    return (pos , pos + len(words) - 1 , label)

In [7]:
LABEL = "Disney"

In [8]:
words = ["disneyland" , "pass" , "billet" , "disney" , "pass" , "adress" , "anniversair",
        "pass" , "parc" , "parc" , 'attract' , 'personnag' , "pass" , 'pass' , "disney" , "disney",
        "disney" , "répons" , "spectacl" , "attract" , "disney" , 'pass' , "compt" , "entr" , 'parc',
        "restaur" , "disneyland" , "beb" , "disneyland" ," soin" ,'enfant',"parc" , "personnag", "disneyland",
         "hôtel" , "âg" , "éven" , "plan" , "parc" , "disneyland" , "classic , fantasy ou dream" , "hôtel",
         "hôtel" , "parc" , "parc" , "disneyland" , "parc" , "villag" ,"check-out" , "pass" , "disneyland",
         "pass" , 'enfant' , "moment" ,"moment" , "déplac" , "déplac" , "attract" , "attract" , "activ" , "activ",
         "coordon" , "ouvertur" , "sécur" , "proxim" , "beb" , "avantag" , "passeport" , "bureau" , "restaur",
         "boutiqu" , "séjour" , "pousset" , 'bas âge' , "acces wif" , "départ"
        ]

TRAIN_DATA = [(train["stem_quest"].iloc[i] , 
           {"entities" : [ localisation(train["stem_quest"].iloc[i] , words[i] , LABEL) ]}
              )
              for i in range(len(words))]
TRAIN_DATA += [(train["stem_quest"].iloc[i], {"entities" :  []}) for i in range(76 , 111)]
 



In [9]:
output_dir = "C:/Users/Benco/Documents/chatBot_disney"


In [124]:
plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", pathlib.Path),
    n_iter=("Number of training iterations", "option", "n", int),
)

<function plac_core.annotations.<locals>.annotate(f)>

In [26]:
def main(model=None, output_dir=None, n_iter=100):
    random.seed(0)
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("fr")  # create blank Language class
        print("Created blank 'fr' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    ner.add_label("Disney")
    ##mess nothing
    ner.add_label("VEGETABLE")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimize = nlp.resume_training()
    move_names = list(ner.move_names)
    #get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if(pipe != "ner")]
    while nlp.pipe_names == ["tagger" , "parser" , "ner"]: #only train ner
        nlp.disable_pipes(*other_pipes)
        sizes = compounding(1.0 , 4.0 , 1.001)
        #batch up
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA , size = sizes)
            losses = {}
            for batch in batches:
                texts , annotations = zip(*batch)
                nlp.update(texts , annotations , sgd = optimize , drop = 0.35 , losses = losses)
            print("Losses" , losses)
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    for text, _ in TRAIN_DATA:
        doc = nlp2(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    test_text = stem_text('et on achète le pain ou ?')
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label , ent.text)
        
    if output_dir is not None:
        output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
        nlp.meta["name"] = "new_model_name"
        nlp.to_disk(output_dir)
        print("Saved model to" , output_dir)
    #test the saved model
    print("Loading from" , output_dir)
    nlp2 = spacy.load(output_dir)
    #check the classes have loaded back consis
    assert nlp2.get_pipe("ner").move_names == move_names
    doc2 = nlp2(test_text)
    for ent in doc2.ents:
        print(ent.label , ent.text)
#save model to output directory
    

In [27]:
main(model = "C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5" , output_dir = output_dir)

Loaded model 'C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5'
Losses {'ner': 1095.7731546511832}
Losses {'ner': 934.9953700358476}
Losses {'ner': 890.5119107285877}
Losses {'ner': 906.7485730136284}
Losses {'ner': 852.9999429074815}
Losses {'ner': 858.7525098717881}
Losses {'ner': 790.9843814342134}
Losses {'ner': 857.3191888498768}
Losses {'ner': 853.7036814013227}
Losses {'ner': 848.5764848834042}
Losses {'ner': 861.407534239197}
Losses {'ner': 815.2668902937176}
Losses {'ner': 849.9819708114192}
Losses {'ner': 854.0952256819346}
Losses {'ner': 822.3861806461355}
Losses {'ner': 824.7419423013926}
Losses {'ner': 861.1788164612954}
Losses {'ner': 843.9031866565347}
Losses {'ner': 810.8673251956934}
Losses {'ner': 846.9992763027549}
Losses {'ner': 807.692088027994}
Losses {'ner': 877.6222118735313}
Losses {'ner': 800.4432575106621}
Losses {'ner': 758.1764730215073}
Losses {'ner': 807.8273440003395}
Losses {'ner': 808.810177760286}
Losses {'ner': 779.818

Entities []
Tokens [('comment', '', 2), ('puis', '', 2), ('-', '', 2), ('j', '', 2), ('vérifi', '', 2), ('que', '', 2), ('ma', '', 2), ('souscript', '', 2), ('a', '', 2), ('été', '', 2), ('enregistr', '', 2), ('?', '', 2)]
Entities []
Tokens [('où', '', 2), ('puis', '', 2), ('-', '', 2), ('j', '', 2), ('chang', '', 2), ('ou', '', 2), ('nourr', '', 2), ('mon', '', 2), ('beb', '', 2), ('ou', '', 2), ('réchauff', '', 2), ('de', '', 2), ('bib', '', 2), ('?', '', 2)]
Entities []
Tokens [('le', '', 2), ('prix', '', 2), ('avec', '', 2), ('un', '', 2), ('pai', '', 2), ('compt', '', 2), ('ou', '', 2), ('mensuel', '', 2), ('est-il', '', 2), ('le', '', 2), ('mêm', '', 2), ('?', '', 2)]
Entities []
Tokens [('combien', '', 2), ('dois', '', 2), ('-', '', 2), ('j', '', 2), ('pai', '', 2), ('de', '', 2), ('frais', '', 2), ('de', '', 2), ('résili', '', 2), ('si', '', 2), ('je', '', 2), ('quitt', '', 2), ('mon', '', 2), ('oper', '', 2), ('actuel', '', 2), ('alor', '', 2), ('que', '', 2), ('je', '', 2), 

Entities []
Tokens [('comment', '', 2), ('déclar', '', 2), ('un', '', 2), ('chang', '', 2), ("d'", '', 2), ('adress', '', 2), ('ou', '', 2), ('autr', '', 2), ('situat', '', 2), ('?', '', 2)]
Entities []
Tokens [('lor', '', 2), ('de', '', 2), ('mon', '', 2), ('inscript', '', 2), ('chez', '', 2), ('fre', '', 2), (',', '', 2), ('j', '', 2), ('’', '', 2), ('ai', '', 2), ('demand', '', 2), ('à', '', 2), ('conserv', '', 2), ('mon', '', 2), ('numéro', '', 2), ('de', '', 2), ('mobil', '', 2), ('actuel', '', 2), ('(', '', 2), ('portabl', '', 2), (')', '', 2), (',', '', 2), ('dois', '', 2), ('-', '', 2), ('j', '', 2), ('résili', '', 2), ('mon', '', 2), ('abon', '', 2), ('chez', '', 2), ('mon', '', 2), ('ancien', '', 2), ('oper', '', 2), ('?', '', 2)]
Entities []
Tokens [('comment', '', 2), ('savoir', '', 2), ('si', '', 2), ('le', '', 2), ('chaussur', '', 2), ('de', '', 2), ('mon', '', 2), ('enfant', '', 2), ('sont', '', 2), ('trop', '', 2), ('grand', '', 2), ('?', '', 2)]
Entities []
Tokens [('p

Tokens [('comment', '', 2), ('profit', '', 2), ('d', '', 2), ('’', '', 2), ('un', '', 2), ('deuxiem', '', 2), ('(', '', 2), ('ou', '', 2), ('troisiem', '', 2), (',', '', 2), ('ou', '', 2), ('quatriem', '', 2), (')', '', 2), ('`', '', 2), ('`', '', 2), ('forf', '', 2), ('fre', '', 2), ("''", '', 2), ('à', '', 2), ('15,99', '', 2), ('€', '', 2), ('par', '', 2), ('mois', '', 2), ('?', '', 2)]
Entities []
Tokens [("j'", '', 2), ('ai', '', 2), ('une', '', 2), ('question', '', 2), ('mais', '', 2), ('je', '', 2), ('ne', '', 2), ('trouv', '', 2), ('pas', '', 2), ('la', '', 2), ('répons', '', 2), ('dan', '', 2), ('cet', '', 2), ('rubriqu', '', 2), (',', '', 2), ('que', '', 2), ('puis', '', 2), ('-', '', 2), ('j', '', 2), ('fair', '', 2), ('?', '', 2)]
Entities []
Tokens [('mon', '', 2), ('enfant', '', 2), ('peut', '', 2), ('-', '', 2), ('il', '', 2), ('port', '', 2), ('de', '', 2), ('chaussur', '', 2), ('à', '', 2), ('crampon', '', 2), ('destin', '', 2), ('à', '', 2), ("d'", '', 2), ('autr', ''

Tokens [('comment', '', 2), ('savoir', '', 2), ('si', '', 2), ('le', '', 2), ('chaussur', '', 2), ('de', '', 2), ('mon', '', 2), ('enfant', '', 2), ('sont', '', 2), ('trop', '', 2), ('grand', '', 2), ('?', '', 2)]
Entities []
Tokens [('peut', '', 2), ('-', '', 2), ('on', '', 2), ('port', '', 2), ('de', '', 2), ('chaussur', '', 2), ('nik', '', 2), ('san', '', 2), ('chausset', '', 2), ('?', '', 2)]
Entities []
Tokens [('l', '', 2), ('’', '', 2), ('utilis', '', 2), ('de', '', 2), ('perch', '', 2), ('télescop', '', 2), (',', '', 2), ('utilis', '', 2), ('not', '', 2), ('pour', '', 2), ('le', '', 2), ('self', '', 2), (',', '', 2), ('est-el', '', 2), ('autoris', '', 2), ('dan', '', 2), ('le', '', 2), ('parc', '', 2), ('à', '', 2), ('them', '', 2), ('de', '', 2), ('disneyland', '', 2), ('paris', '', 2), ('?', '', 2)]
Entities []
Tokens [('je', '', 2), ('suis', '', 2), ('handicap', '', 2), ('.', '', 2), ('est', '', 2), ('-ce', '', 2), ('que', '', 2), ('je', '', 2), ('bénéfic', '', 2), ("d'", ''

In [90]:
doc1 = nlp(train["lemmas_quest"].iloc[80])
doc2 = nlp(train["lemmas_quest"].iloc[98])
doc1.similarity(doc2)

C:\Users\Benco\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


0.32501200593594537

In [6]:
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']

In [7]:
X_train = train[train.target == 1]

In [41]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words = sw,
                            ngram_range=(1, 1),
                            min_df = 3,
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')

In [42]:
X_train_vectorized_tfidf = vectorizer.fit_transform(X_train["stem_quest"])# compléter ici

In [54]:
query = [stem_text(train["question"][69])]
query_vector = vectorizer.transform(query)
pd.DataFrame(query_vector.toarray(), columns=vectorizer.get_feature_names())

,acces,annuel,attract,beb,billet,chang,dan,disney,disneyland,dois,...,plus,possibl,prix,puis,rendr,servic,si,spécial,trouv,visit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.488838,0.408051,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.626945,0.0


In [52]:
query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector)) ## on enlève leur format qui n'est pas bon --> vecteur
query_corpus_sim

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.57364585,
       0.70033383, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.36763093, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.70741969, 0.        , 0.        ,
       0.        , 0.        , 0.52063869, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.31382392, 0.        , 0.        , 0.        , 0.        ,
       0.40483297, 0.        , 0.        , 0.2996493 , 0.     

In [12]:
indice = np.argsort(query_corpus_sim) ## ordre croissant
indice

array([ 0, 53, 52, 50, 49, 48, 47, 54, 46, 44, 43, 42, 41, 39, 38, 45, 74,
       55, 57, 73, 72, 71, 70, 69, 67, 56, 65, 63, 62, 61, 60, 59, 58, 64,
       36, 37, 34,  2,  5,  6,  7,  8,  9, 10, 11, 14, 15, 16, 35, 18, 19,
       20, 17, 30, 33, 32, 31, 29, 28, 27, 75, 26, 25, 23, 22,  4, 21, 24,
       12,  3, 13, 51, 68, 66,  1, 40], dtype=int64)

In [204]:
train["stem_quest"][indice[-1]]

'je possed actuel un passeport annuel classic , fantasy ou dream , suis-j oblig ( e ) de le chang pour un nouveau pass annuel ?'

In [207]:
np.count_nonzero(query_corpus_sim) > 2

True

In [55]:
class Agent:
    def __init__(self , data , stop_words , seuil):
        self.data = data
        self.stop_words = stop_words
        self.stop_words += ["être" , "avoir" , "voir" , "choisir" , "quel" , "comment" , "?"]
        self.seuil = seuil
        
    def classification(self , query):
        vectorizer = TfidfVectorizer(lowercase=True, stop_words = self.stop_words,
                            ngram_range=(1, 2),
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')
        X_train_vectorized_tfidf = vectorizer.fit_transform(self.data["stem_quest"])
        query = [stem_text(query)]
        query_vector = vectorizer.transform(query)
        query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector))
        nb_non_zeros = np.count_nonzero(query_corpus_sim)
        
        if (nb_non_zeros >= self.seuil):
            return 1
            #return {"classif" : 1 , "doc_most_similar" : np.argsort(query_corpus_sim)[-nb_non_zeros:]}
        else:
            return 0


In [56]:
agent = Agent(X_train , sw , 10)
agent.classification(train["question"][105])

1

In [58]:
cpt = 0
for i in range(76):
    if(agent.classification(train["question"][i]) == 0):
        cpt += 1
cpt/(76)

0.09210526315789473